# Emerging Technologies Tasks

My solutions to the Emerging Technologies tasks assessment.

## Task 1: Square Root of 2

>*Write a Python function called `sqrt2` that calculates and prints to the screen the square root of 2 to 100 decimal places. Your code should not depend on any module from the standard library or otherwise. You should research the task first and include references and a description of your algorithm.*

### Algorithm Description

Before discussing the algorithm used to solve this problem, it is necessary to first provide some background on how floating point numbers are stored on a computer.

#### Floating Point Numbers

Floating point numbers are used to represent fractional values and are stored in a computer as base 2 (binary) fractions. Most decimal fractions cannot be accurately represented as binary fractions and as a result, decimal floating-point numbers are only approximated by the binary floating-point numbers actually stored in the machine \[1\]. For instance, the summation of $0.1$ and $0.2$ yields a result of $0.3$ in decimal. However, the result of this computation cannot be accurately represented in binary (see below).

In [1]:
0.1 + 0.2

0.30000000000000004

The IEEE standard defines 32-bit and 64-bit floating-point representations. The 32-bit (single-precision) format consists of a sign bit, an 8-bit exponent and 23 bits of mantissa. The 64-bit (double-precision) format consists of a sign bit, an 11-bit exponent and 52 bits of mantissa. Floating-point arithmetic on computers is therefore inherently inexact as the 24 bits of mantissa in a 32-bit floating-point number can only represent approximately 7 significant decimal digits (for doubles this is about 16 significant decimal digits \[2\]\[3\]). If a number is not exactly representable, then it must instead be approximated \[2\].

Python's float type has double-precision \[1\], and it is therefore not possible to store a number with 100 decimal places as a float. However, in Python integers have unlimited precision \[4\], so one approach to the given problem is to store all the significant digits of $\sqrt2$ as an integer. Finding the square root of a given number $p$ as an integer can be achieved by calculating the *integer square root* of $p \times 10^{2 \times q}$, where $q$ is the number of digits after the decimal point, in this case 100.

#### Finding the Integer Square Root using Newton's Method

The integer square root of a number is the greatest positive integer less than or equal to the square root of that number, and can be found using Newton's method \[5\]. Newton's algorithm works by producing successively better approximations of the roots (or zeroes) of a real-valued function \[6\]. The algorithm starts by taking some approximation of the square root ($x_{k}$) and then iteratively applying the below formula until $|x_{{k + 1}} - x_{{k}}| < 1 $ \[5\].

$$ x_{k+1} = \frac{1}{2}\left(x_{k} + \frac{n}{x_{k}}\right) $$

Assuming the value $x_{k}$ is close enough to the root, then $x_{k+1}$ will be even closer to the desired root. The rate of convergence is quadratic, meaning that the number of digits in the approximate value ($x_{k}$) doubles with each iteration [7].

### Solution

Below is my solution to task one. I start by defining a function, `isqrt()` that calculates the integer square root of a number. I then define the `sqrt2()` function which uses `isqrt()` to calculate the square root of 2 to 100 decimal places and print the result to the screen.

In [2]:
def isqrt(n: int) -> int:
    """
    Calculates the integer square root of `n` using Newton's method.
    :return: the integer square root of `n`.
    :raises ValueError: If `n` is not a positive integer.
    """
    # Code adapted from: user448810 - https://stackoverflow.com/a/15391420
    
    if (n <= 0):
        raise ValueError("n must be a positive integer")
    
    # Set an initial guess for k
    k = n // 2

    while True:
        # Cache the previous value of k
        prev_k = k
        
        # Apply the formula
        k = (k + (n // k)) // 2

        # Check loop stop condition
        if (prev_k - k) < 1:
            break
        
    return k

In [3]:
def sqrt2():
    """
    Calculates and prints to the screen the square root of 2 to 100 decimal places.
    """
    # Set the number of places to show after the decimal point
    precision = 100

    # Get the integer square root of 2 x 10^200
    # Reference: Eugene Yarmash - https://stackoverflow.com/a/32651586
    result = str(isqrt(2 * 10 ** (2 * precision)))

    # Insert the decimal point
    result = result[:1] + '.' + result[1:]

    # Output the result
    print(result)

### Testing

In order to illustrate the function's accuracy, we can compare the output of the standard library's `math.sqrt()` function with the above `sqrt2()` function.

In [4]:
from math import sqrt

print(sqrt(2))
sqrt2()

1.4142135623730951
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727


We'll find that both functions appear to output the same number, but since `math.sqrt()` returns a float it can only represent the number up to about 16 decimal places. We can however do better by comparing our result to that given by the `sqrt()` function provided by Pyhon's built-in `decimal` module. Doing so, we find both functions produce the same value.

In [5]:
from decimal import (
    getcontext,
    Decimal
)

getcontext().prec = 101
result = Decimal(2).sqrt()

print(result)
sqrt2()

1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727


## Task 2: Chi-squared Test for Independence

>*The Chi-squared test for independence is a statistical hypothesis test like a t-test. It is used to analyse whether two categorical variables are independent. The [Wikipedia](https://en.wikipedia.org/w/index.php?title=Chi-squared_test&oldid=983024096) article gives the table below as an example, stating the Chi-squared value based on it is approximately 24.6. Use `scipy.stats` to verify this value and calculate the associated p value. You should include a short note with references justifying your analysis in a markdown cell.*

|   | A | B | C | D | Total |
| - | - | - | - | - | ----- |
| White collar | 90 | 60 | 104 | 95 | 349 |
| Blue collar  | 30 | 50 | 51 | 20 | 151 |
| No collar    | 30 | 40 | 45 | 35 | 150 |
| **Total**        | 150 | 150 | 200 | 150 | 650 |

### Solution

In [6]:
from scipy.stats import chi2_contingency
import numpy as np

# Express the values in the table as a NumPy array
table = np.array ([
    [90, 60, 104, 95],
    [30, 50, 51, 20],
    [30, 40, 45, 35],
])

result, p, _, _ = chi2_contingency(table)

print(f"Result: {round(result, 1)}")
print(f"p value: {p}")

Result: 24.6
p value: 0.0004098425861096696


### Justification & Analysis

The `chi2_contingency` function provided by `scipy.stats` is used to compute the chi-square statistic and *p* value of the frequencies in the given contingency table [8]. In its simplest form, the function takes an `array_like` object containing the observed frequencies in each category. In the code above, we express the values in the table as a two dimensional NumPy array and pass that into the `chi2_contingency` function. The function returns a 4-tuple [8], however for the purposes of this task we are only interested in the first two values, the test statistic and the *p* value.

Using the above function, we find the test statistic value is approximately 24.6, thus verifying that the value from the Wikipedia article is accurate.

#### How it Works

The chi-squared test of independence compares our sample data in the contingency table to the distribution of values we'd expect if the null hypothesis is correct (the null hypothesis being that the variables are independent) [9]. This means in order to calculate the test statistic, we must first construct the contingency table we’d expect to see if the null hypothesis is true [9]. The expected value ($E$) of a given cell in the table is found using the below formula [9] and is calculated automatically by the `chi2_contingency` function [8].

$$
E = \frac{(row\ total)(column\ total)}{sample\ size}
$$

The test statistic ($X^2$) then, is calculated by finding the sum of the squared differences between each pair of observed and expected values [9]. It is expressed by the below formula, where $O$ is the observed value and $E$ is the expected value [9]. Applying this formula over all of the cells in our table and then summing the result calculates the test statistic [9], which is the value ultimately returned by the `chi2_contingency` function. 

$$
X^2 = \sum{\frac{(O - E)^2}{E}}
$$

## References

1. [*Floating Point Arithmetic: Issues and Limitations*](https://docs.python.org/3/tutorial/floatingpoint.html). The Python 3 Standard Library Documentation.
2. [*The Perils of Floating Point*](http://www.lahey.com/float.htm). Bush, Bruce M. Lahey Computer Systems, Inc.
3. [*Double-precision floating-point format*](https://en.wikipedia.org/wiki/Double-precision_floating-point_format). Wikipedia, the free encyclopedia.
4. [*Built-in Types*](https://docs.python.org/3/library/stdtypes.html). The Python 3 Standard Library Documentation.
5. [*Integer square root*](https://en.wikipedia.org/wiki/Integer_square_root). Wikipedia, the free encyclopedia.
6. [*Newton's Method*](https://en.wikipedia.org/wiki/Newton%27s_method). Wikipedia, the free encyclopedia.
7. [*Newton's method for finding roots*](https://cp-algorithms.com/num_methods/roots_newton.html). CP-Algorithms.
8. [*scipy.stats.chi2_contingency*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html). SciPy.org documentation.
9. [*How the Chi-Squared Test of Independence Works*](https://statisticsbyjim.com/hypothesis-testing/chi-squared-independence/). Frost, Jim. Statistics By Jim.